Plan for steps in this processing code
1. Define a base grid
2. Load all the tif files - know how
4. Load the shp files - know how
5. Project the shp files to tif - at the resolution / bounds, etc with the base grid - know how
6. Generate the train deposit / occurence tif files - know how
6. Unify all the tif data - know how

In [ ]:
import numpy as np
from tqdm import tqdm
import rasterio

import utilities as utils

RAW_DATA_DIR = "data/LAWLEY22-RAW/"
DERIV_DATA_DIR = "data/LAWLEY22-DERIV/"

In [ ]:
tifs, shps = utils.get_input_var_files("USCAN")

Loads the raster data

In [ ]:
rasters = utils.load_rasters(tifs, rasters_path=RAW_DATA_DIR, verbosity=1)

We'll need to upsample rasters that have too low a resolution

In [ ]:
target_resolution = (0.008, 0.008)
rasters = utils.resample_rasters(target_resolution, rasters, tifs)

Loads rasters of the vector data if available; otherwise generates them

In [ ]:
try:
    rasters += utils.load_rasters(shps, rasters_path=RAW_DATA_DIR, verbosity=1)
except rasterio.RasterioIOError:
    base_raster = rasters[-1] # defaults to intermediate resolution raster
    vectors = utils.load_vectors(shps, vectors_path=RAW_DATA_DIR, verbosity=0)
    pbar = tqdm(zip(shps, vectors))
    for shp, vector in pbar:
        pbar.set_description(f"Processing {shp}")
        utils.proximity_raster_of_vector_points(base_raster, shp, vector)
    rasters += utils.load_rasters(shps, rasters_path=RAW_DATA_DIR, verbosity=1)

Loads the base grid for all data if available; otherwise generates it

In [ ]:
grid_cell_ids = utils.generate_s2_grid(rasters, DERIV_DATA_DIR, "s2_grid_uscan")

Initialize the datacube

In [ ]:
datacube = utils.init_datacube({"s2_cell_id": grid_cell_ids}, ["s2_cell_center", "s2_cell_poly"] + tifs + shps, verbosity=1)

Load file with Deposits and Occurrences

In [ ]:
df_dep, df_occ = utils.process_raw_deposit_file('GeologyMineralOccurrences_USCanada_Australia.csv', csv_path='data/LAWLEY22-RAW/', region='USCanada', dep_grp='MVT')

Adding MVT_Deposit, MVT_Occurrence columns to datacube

In [ ]:
datacube, notrecogdep = utils.mvt_dep_occur_to_s2cells(datacube, df_dep, colname='MVT_Deposit')
datacube, notrecogocc = utils.mvt_dep_occur_to_s2cells(datacube, df_occ, colname='MVT_Occurrence')

Add neighbors column to datacube

In [ ]:
datacube = utils.neighbor_deposits(datacube, deptype='MVT')

Populate the datacube using as many process as available CPUs

In [ ]:
datacube = utils.populate_datacube(datacube, tifs+shps)

Final filtering of s2 cells that contain no data

In [ ]:
print(f"Removing {np.count_nonzero(np.isnan(datacube.loc[:, tifs+shps].values).all(axis=1))} s2 cells that have no geophysical data")
datacube = datacube[~np.isnan(datacube.loc[:, tifs+shps].values).all(axis=1)]

Store the datacube for future use

In [ ]:
datacube.to_csv(f"{DERIV_DATA_DIR}datacube_uscan.csv", index=False)

In [ ]:
# datacube.info()

We'll rasterize the datacube to visualize it and confirm the output

In [ ]:
# import pandas as pd
# datacube = pd.read_csv(f"{DERIV_DATA_DIR}datacube_uscan.csv")
# from shapely import wkt
# datacube["s2_cell_poly"] = datacube["s2_cell_poly"].apply(wkt.loads)

In [ ]:
# continent = "uscan"

# bounds = {'left':-179.8, 'bottom':24.5, 'right':-52.6, 'top':83.15}

# resolution = (0.02, 0.02)

# nodata_value = rasters[0].nodata

# height = int((bounds["top"]-bounds["bottom"]) / resolution[0])
# width = int((bounds["right"]-bounds["left"]) / resolution[1])

# tif_tf = rasterio.transform.from_bounds(
#     bounds["left"],
#     bounds["bottom"],
#     bounds["right"],
#     bounds["top"],
#     width,
#     height
# )

# tiff_meta = {
#     "driver": 'GTiff',
#     "height": height,
#     "width": width,
#     "count": 1,
#     "dtype": "float64",
#     "crs": rasterio.CRS.from_epsg(4326),
#     "transform": tif_tf,
#     "nodata": nodata_value,
#     "compress": "lzw",
# }

# # utils.rasterize_datacube(datacube, tiff_meta, DERIV_DATA_DIR, continent.lower().replace(" ","-"))
# utils.visualize_datacube(datacube, tiff_meta)